In [ ]:
import utils
import os
import pandas as pd

In [ ]:

execution_time = pd.Timestamp.now()


TRAIN_CONFIG = {
    "training_path":"./data/final_images/train",
    "validation_path":"./data/final_images/validation", 
    "test_path":"./data/final_images/test",
    "network":"unet_pluplus", # valid network: unet, deeplabv3, attention_unet, attention_resunet, unet_pluplus
    "save_model":True,
    "downscaling_factor":32, #If specified you can keep track of performance in the excel file
    "n_classes":5,
    "batch_size": 16,
    "epochs": 1,
    "validation_split":0.2,
    "test_split":0.1,
    "learning_rate":0.001,
    "early_stopping_patience":50,
    "lr_on_plateau_patience":25,
    "lr_mod_factor":0.1,
    "execution_time": execution_time.strftime("%Y%m%d_%H%M%S"),
    "results_folder": "results_" + execution_time.strftime("%Y%m%d_%H%M%S"),
    "verbose": 1 if os.name=="nt" else 0 # dissables verbose if we run on Snellius
}

## Data Preprocessing

Extract the labels from its corresponding label file

In [ ]:
utils.generate_training_labels(images_path="./data/66-4/"
                               , labels_path="./data/GeoJson"
                               , resolution=32
                               , validation_split=TRAIN_CONFIG["validation_split"]
                               , test_split=TRAIN_CONFIG["test_split"])

Builds mossaics with image labels

In [ ]:
utils.build_mossaic(labels_source_directory="./data/train_labels" 
                    , mossaic_destination_directory="./data/mossaics/training_mossaics"
                    , matrix_size=1024
                    , data_augmentation=False
                    , patchify_large_labels=True
                    )

In [ ]:
utils.build_mossaic(labels_source_directory="./data/validation_labels" 
                    , mossaic_destination_directory="./data/mossaics/validation_mossaics"
                    , matrix_size=1024
                    , data_augmentation=False
                    , patchify_large_labels=True
                    )

utils.build_mossaic(labels_source_directory="./data/test_labels"
                    , mossaic_destination_directory="./data/mossaics/test_mossaics"
                    , matrix_size=1024
                    , data_augmentation=False
                    , patchify_large_labels=True
                    )

Slices the mossaics into the desired size for training

In [ ]:
utils.prepare_training_images(mossaic_directory="./data/mossaics/training_mossaics"
                              , destination_directory= "./data/final_images/train"
                              , slice_size=(128, 128)
                              , desired_background_color = [240, 239, 241]
                              , tolerance=0.05)

utils.prepare_training_images(mossaic_directory="./data/mossaics/validation_mossaics"
                              , destination_directory= "./data/final_images/validation"
                              , slice_size=(128, 128)
                              , desired_background_color = [240, 239, 241]
                              , tolerance=0.05)

utils.prepare_training_images(mossaic_directory="./data/mossaics/test_mossaics"
                              , destination_directory= "./data/final_images/test"
                              , slice_size=(128, 128)
                              , desired_background_color = [240, 239, 241]
                              , tolerance=0.05)

## Model Training

In [ ]:
execution_time = pd.Timestamp.now()

test_set = [50, 100, 200, 300]
networks = ["unet", "deeplabv3", "attention_unet", "attention_resunet", "unet_pluplus"]

# for i in range(len(test_set)):
#     print(f"Testing {test_set[i]} epochs")
#     for network in networks:
TRAIN_CONFIG = {
    "training_path":"./data/final_images/train",
    "validation_path":"./data/final_images/validation", 
    "test_path":"./data/final_images/test",
    "network":"unet",
    "save_model":True,
    "downscaling_factor":32, #If specified you can keep track of performance in the excel file
    "n_classes":5,
    "batch_size": 16,
    "epochs": 1000,
    "validation_split":0.1,
    "test_split":0.1,
    "learning_rate":0.001,
    "early_stopping_patience":10,
    "lr_on_plateau_patience":11,
    "lr_mod_factor":0.1,
    "execution_time": execution_time.strftime("%Y%m%d_%H%M%S"),
    "results_folder": "results_" + execution_time.strftime("%Y%m%d_%H%M%S"),
    "verbose": 1 if os.name=="nt" else 0 # dissables verbose if we run on Snellius
}
utils.train_model(TRAIN_CONFIG)